# [MCB 32]: Lab 04 - Muscle Excitabilitty and Contractility
 
---

### Professor Robin Ball

We will introduce you to data analysis using Python and Jupyter notebooks using the data you collect in this lab. You will receive more information about this during lab.

*Estimated Time: ~3 Hours*

---

### Table of Contents

I: [Introduction to Tables + Visualization](#section 1)<br>

II: [Background](#section 2)<br>

III: [Stimulating Skeletal Muscle](#section 3)<br>

IV: [Twitch Size Versus Stimulus Strength (Recruitment)](#section 4)<br>

V: [Summation and Tetanus (Rate Coding)](#section 5)<br>

VI: [Potentiation by Prior Contractions](#section 6)

In [4]:
# Import pandas, a data science library, and matplotlib for plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datascience import *

## I. Introduction to Tables
---

In this notebook, you will learn about tables in the datascience module and methods associated with them.
This notebook assumes that you have already completed the "Introduction to Data Science" notebook.

---

**Topics Covered:**
- Learn how to import and create new Tables.
- Learn how to access certain rows, columns, and values.
- Learn various methods of finding new information from a table of data.

### Creating Tables

When dealing with a collection of things with multiple attributes, it can be useful to put the data in a _table_.  Tables are a nice way of organizing data in a 2-dimentional data set. For example, take a look at the table below.

In [5]:
Table.read_table('http://inferentialthinking.com/notebooks/nba_salaries.csv')

PLAYER,POSITION,TEAM,'15-'16 SALARY
Paul Millsap,PF,Atlanta Hawks,18.6717
Al Horford,C,Atlanta Hawks,12
Tiago Splitter,C,Atlanta Hawks,9.75625
Jeff Teague,PG,Atlanta Hawks,8
Kyle Korver,SG,Atlanta Hawks,5.74648
Thabo Sefolosha,SF,Atlanta Hawks,4
Mike Scott,PF,Atlanta Hawks,3.33333
Kent Bazemore,SF,Atlanta Hawks,2
Dennis Schroder,PG,Atlanta Hawks,1.7634
Tim Hardaway Jr.,SG,Atlanta Hawks,1.30452


This table shows players in the NBA, their position, team, and salary (in millions of dollars) for each player. To create this table, we have drawn the data from the file at <a href="http://inferentialthinking.com/notebooks/nba_salaries.csv">this link</a>, called `nba_salaries.csv`. In general, to import data from a `.csv` file, we write **`Table.read_table("file_name")`.**

However, we can also create our own tables from scratch without having to import data from another file. Let's say we have two arrays, one with a list of fruits, and another with a list of their price at the Berkeley Student Food Collective. Then, we can create a new `Table` with each of these arrays as columns with the `with_columns` method:

In [6]:
fruit_names = make_array("Apple", "Orange", "Banana")
fruit_prices = make_array(1, 0.75, 0.5)
fruit_table = Table().with_columns("Fruit", fruit_names,
                                  "Price ($)", fruit_prices)
fruit_table

Fruit,Price ($)
Apple,1
Orange,0.75
Banana,0.5


The **`with_columns`** method takes in pairs of column labels and arrays, and creates a new table with each array as a column of the table. Finally, to create a new table (with no columns or rows), we simply write

In [7]:
empty_table = Table()
empty_table

As a summary, here are the functions we can use to manipulate tables:
    
|Name|Example|Purpose|
|-|-|-|
|`Table`|`Table()`|Create an empty table, usually to extend with data|
|`Table.read_table`|`Table.read_table("my_data.csv")`|Create a table from a data file|
|`with_columns`|`tbl = Table().with_columns("N", np.arange(5), "2*N", np.arange(0, 10, 2))`|Create a copy of a table with more columns|
|`column`|`tbl.column("N")`|Create an array containing the elements of a column|
|`sort`|`tbl.sort("N")`|Create a copy of a table sorted by the values in a column|
|`where`|`tbl.where("N", are.above(2))`|Create a copy of a table with only the rows that match some *predicate*|
|`num_rows`|`tbl.num_rows`|Compute the number of rows in a table|
|`num_columns`|`tbl.num_columns`|Compute the number of columns in a table|
|`select`|`tbl.select("N")`|Create a copy of a table with only some of the columns|
|`drop`|`tbl.drop("2*N")`|Create a copy of a table without some of the columns|
|`take`|`tbl.take(np.arange(0, 6, 2))`|Create a copy of the table with only the rows whose indices are in the given array|


---



## II. Background  <a id='section 2'></a>

Inhale. Smile. Sing a song. Walk to a cafe.  Bite into a croissant.  All of these actions depend on skeletal muscles.  The muscles may contract quickly, slowly, gently, or with great force.  They may act for a brief instant or continue for hours.  You may control them voluntarily or they may act reflexively without you even being aware of them.  In this lab, you will simulate the effect of motor neurons on muscles by applying controlled electrical stimuli to a muscle in your hand. To measure the effect of the stimuli, you will record the motion of a finger controlled by that muscle.  The questions you will address in this investigation include:

> 1. How does varying the strength of one stimulus (voltage applied) affect the muscle twitch?
2. How does a muscle respond to changes in stimulus frequency at constant stimulus strength?
3. How do repeated contractions alter the characteristics of a muscle twitch?

### Overview
Skeletal muscle is a tissue that is specialized for movement and is associated with voluntary movements of the head, limbs and torso.  Skeletal muscle is composed of many muscle fibers organized into motor units.  In vertebrates, each muscle fiber is innervated by only one motor neuron,  and a single motor neuron typically innervates many muscle fibers (tens to thousands), all of the same fiber type.  The combination of a single motor neuron plus all of the muscle fibers it controls is called a **motor unit**.  When a motor neuron fires an action potential, all fibers in its motor unit are stimulated simultaneously in an all-or-none contraction called a **twitch**.
 
### Recruitment of Motor Units
There are many motor units controlling a single muscle (made up of multiple muscle fibers).  More and more of these units can be recruited in order to increase the strength of contraction. Such recruitment is referred to as **spatial summation** or **recruitment of motor units**. 

A second way to generate more force is to *stimulate motor units more frequently* (see Fig. 1).  This allows more tension to be developed than does a single twitch, because the second muscle twitch can be initiated before the first twitch has completely relaxed.  This second twitch is then added to the first twitch.  Such increased firing of motor units is referred to as **tetanus** or **temporal summation** (Fig. 1C).

<figure>
  <img src= 'img/fig1.png' width='60%'></img>
  <figcaption>**Fig. 1**: Typical results showing effects of different frequencies of stimulation on contractile strength.</figcaption>
</figure>

To understand the **summation of twitches** during such rate coding, it is necessary to understand the normal course of events within a muscle fiber during a single twitch.  An action potential causes release of calcium from its internal store (the sarcoplasmic reticulum).  The calcium binds to the regulatory proteins in the fiber, and actin binding sites are uncovered, so that myosin cross-bridges attach, producing force and movement.  The calcium also binds to other proteins in the fiber, including pumps in the sarcoplasmic reticulum (SR) that begin to resequester it, again lowering the calcium concentration in the myofibrillar space.  As calcium concentration declines, it comes off the regulatory proteins, inhibition is re-established, and the fiber relaxes as cross-bridges detach.  Typically, the calcium transient lasts only during the first 10-20 milliseconds of a twitch that may last hundreds of milliseconds.  The relaxation process typically takes many tens of milliseconds to a few hundred milliseconds.  Therefore, one mechanism for twitch summation involves stimulating the cells frequently enough that calcium accumulates in the cytosol. 

Human motor units are typically recruited at frequencies between 8 and 11 Hz.  Once recruited, they rate code up to tetanic frequencies, i.e., they increase their contractions as the frequency of stimulations increases.  Thus, most motor units are not producing smooth (fused tetanus) forces during most of our muscle contractions.  Rather, we achieve smooth contractions (Fig. 1C) by adding the forces from many unfused tetani (Fig 1B) together so that the peaks even out (asynchronous recruitment).

---

## III. Stimulating Skeletal Muscle <a id='section 3'></a>

---

**SAFETY WARNING: BEFORE using the stimulator on a human subject, set all the controls properly for the first trial, then have your GSI or UGSI check that you have set it up properly. Observe basic electrical safety precautions.  Keep the table and floor around the equipment free of liquids.  Do not place electrodes on a subject so that the current will pass through their heart.**

---

<figure>
  <img src= 'img/fig2.png' width='60%'></img>
  <figcaption>**Fig. 2**: Stimulator menu bar</figcaption>
</figure>

The stimulator is built into the iWorx box and can be controlled by the stimulator menu bar at the top of the window (Fig 2)

The correct settings should be entered already, but double check that it is set up correctly, so you do not hurt your subject.

Start with the pulse amplitude at 1 Amp (which is actually 1 mA).  The pulse amplitude corresponds to the strength of the stimulus you will be giving to your subject.  You can type in other numbers between 1-15 Amp, but **do not go above 15 Amp**.

#p is the number of pulses in the stimulus train.  Start with 5, but you can change this number if you want to.

W(ms) is the width of a single pulse (the duration of the pulse).  It should always be set to 5 ms.

F(Hz) is the frequency of stimulation.  Start with 1 Hz, which corresponds to one pulse every second.  5 Hz would be the equivalent of 5 pulses in one second.

Whenever you make a change to a stimulator setting, *you must click on the Apply button* on the left in order for the changes to take effect.


#### Question 3.1: 

---

## IV. Twitch Size Versus Stimulus Strength (Recruitment) <a id='section 4'></a>

In this section you will use single pulses from the stimulator to produce action potentials that generate twitches that move your little finger.  As you increase the strength of the stimulus, more and more axons in the motor nerve will reach threshold and fire an action potential.  Thus, by varying stimulus intensity you can observe the effects of recruitment on a single muscle twitch. Stimulation of a muscle may initially feel peculiar, but following proper procedures stimulation is completely safe.  Typical responses you might observe are shown in Fig. 3.

<figure>
  <img src= 'img/fig3.png' width='60%'></img>
  <figcaption>**Fig. 3**: Sample force tracing illustrating increased twitch tension with increased stimulus amplitude.  Stimulus currents are listed above twitches.</figcaption>
</figure>

#### Methods

> 1. First remove all jewelry from your hands and wrist.  Next clean the back of your hand and the lateral side of your hand between your little finger and wrist, using alcohol wipes.
2. Move your pinky finger to the left and right and feel on the side of your hand for the muscle that is controlling that movement.  This is where you want to place the black electrode.  Apply one electrode (red) on the back of the hand and a second electrode (black) on the side of the hand (see Fig. 5A).  Both electrodes should be placed closer to the wrist than to the fingers. The electrical stimulation from the electrodes will trigger contractions of the abductor digiti minimi muscle and movement of the last digit (Fig. 4).  If the wrong part of the hand is moving, then take off the electrodes and try repositioning them.
3. Attach the muscle transducer to the side of your pinky finger and the side of your hand.  Note that the black strip has one side that should be facing away from your hand (see Fig. 5C).  Use tape to secure the strip to your pinky finger at two locations along the finger.  Use tape and/or the Velcro strips to secure the transducer to the side of your hand (Fig. 5B).  Your lab partners will need to help secure the transducer **tightly** against the hand.  Start Labscribe and move your hand or finger to test if the transducer works.
4. Set the stimulator for 1 Hz frequency and start with the pulse amplitude at 1 Amp.  Set the number of pulses to 5 to start with, but you can change that if you think you need more or less.  Remember to click “Apply” after you make any changes.
5. Relax your hand with the palm facing down.  It works well to rest your hand on a book with the pinky just off the edge.  Start stimulation of the muscle by clicking on Record in Labscribe.  If there is no twitch, then turn up the stimulation amplitude in increments of 0.5 until you get a clear deflection in Labscribe that corresponds to the muscle twitch.  If the wrong muscle is twitching, then reposition the stimulating electrodes on the hand.  You may want to try lower down on the hand.
6. Once you have determined the minimum stimulus required to elicit a single twitch (threshold), increase the stimulus amplitude in 1-2 Amp increments from just below threshold until you get to 15 Amp, or an amplitude that is too intense for your subject.  You should produce five twitches at five different intensities.  Put marks in your data, so you know which amplitude was used for each set of data.  You do not have to wait too long or for a specified time in between each stimulation.  You just need to wait until the twitch has returned to baseline. **Do not increase the stimulus amplitude to the point of causing pain.**  Your data might look something like the data in Figure 3.
7. For each muscle twitch (sample in Fig. 6) record the contraction time (time from beginning of twitch to peak contraction), twitch duration (time from beginning of twitch back to baseline), and the twitch tension (maximum force at the peak of the twitch minus the baseline).  You can determine the values from Labscribe by using the double cursors and looking to the upper right corner at the T2-T1 value and the V2-V1 value. You should take the average of five twitches at each intensity and record that average in Table 1 of the worksheet.
8. Repeat experiment with another subject.

<figure>
  <img src= 'img/fig4.png' width='30%'></img>
  <figcaption>**Fig. 4**: Drawing of the left hand muscles (arrow points to abductor digiti minimi muscle.</figcaption>
</figure>

<figure>
  <img src= 'img/fig5.png' width='60%'></img>
</figure>

<figure>
  <img src= 'img/fig6.png' width='60%'></img>
  <figcaption>**Fig. 6**: Measurement of Twitch Characteristics</figcaption>
</figure>

---

## V. Twitch Size Versus Stimulus Strength (Recruitment) <a id='section 5'></a>

By holding stimulus amplitude and duration constant and varying the frequency of stimuli, you can examine the response of the muscle to multiple action potentials.  At low frequencies, the muscle will respond with a series of identical single twitches.  As frequency increases, summation (twitches begin to fuse) will begin to cause larger but jerky contractions, called unfused tetanus.  At a higher frequency, the contraction will reach a maximal value and become a smooth, fused tetanus.  Typical results are shown in Fig. 7.

<figure>
  <img src= 'img/fig7.png' width='60%'></img>
  <figcaption>**Fig. 7**: Sample force tracings illustrating summation, unfused tetanus and fused tetanus.  Stimulus frequencies are listed above twitches (1, 2, 3, 4 Hz on left and 10, 15, 20, 25, 30 Hz on right).</figcaption>
</figure>

#### Methods

> 1. Based on the "twitch duration" measured in the prior experiment in Table 1, determine the number of contractions that could occur in one second (1sec/twitch duration = predicted summation frequency).  Record this value in Table 2 in the worksheet. Later you will compare the predicted and measured frequencies.
2. Based on the "contraction time" measured in the prior experiment determine the frequency at which fused tetanus is likely to occur (1sec/contraction time = predicted fused tetanus frequency).  Record this value in Table 3 in the worksheet. You will now collect data to compare your predicted values with the actual values.
3. Set the pulse number to 10 and use a stimulus amplitude that produces a nice medium sized twitch. Start at 1 Hz and stimulate the muscle.  Allow the muscle to relax for two seconds and then increase the frequency to 2 Hz and stimulate again.  
4. Repeat this sequence of activations for 3, 4, 5, 6, 7, 8, 9, and 10 Hz.  Remember to put in marks for each frequency, so it is easy to find your data again.
5. From the force tracing try to identify the stimulus frequency where the twitches begin to sum together (the force never gets completely back to the resting value between twitches).  At summation the force tracing often looks like a sharp “V” instead of there being a flat line in between contractions. A second way to identify the summation frequency is when the difference between the peak and resting force of a twitch begins to decrease (see Figure 7).  In Table 2 fill in your measured summation frequency. 
6. Now increase the pulse number to 20 and increase the frequency from the summation frequency in 3-5 Hz increments (2 second recovery) until the twitches are completely fused.  The student who is controlling the frequency on the stimulator should be watching the screen to determine the fusion frequency.  Note that as frequency is increased the rapid contractions begin to fatigue the muscle and can become painful. Be kind to your subjects.  In Table 3 fill in your measured fused tetanus frequency.  How does this compare to your prediction? 
7. Measure the maximum twitch tension at each frequency and enter in Table 4.  

#### Signal Saturation

Similar to the case with the single stimulus, you need to make sure that the signal does not saturate.  If saturation does occur the signal will be flat and you should decrease the stimulus voltage.

---

## VI. Potentiation by Prior Contraction <a id='section 6'></a>

**Only do this experiment if your subject is okay with receiving a high frequency stimulation.  If your subject has been experiencing pain, then do not do this part (but answer the question in the worksheet).

Often people do a "warm-up" (e.g., jog), where the muscle is repeatedly contracted, to prepare for a sporting event.  There are numerous benefits to a warm-up, but do you think that performing repeated contractions changes the twitch tension (indicating that warming up may be beneficial for the muscles)? How long will you need to electrically stimulate the muscle to produce a change in tension (if present) without producing fatigue?  Do you think the stimulation frequency during the activation period has an effect on post-tetanic potentiation?

#### Methods
> 1. Start by producing several twitches at a frequency of 1 Hz using the same stimulus amplitude as in the previous experiments (see Fig. 8). 
2. Increase the frequency to 10 Hz for 20 seconds (figure out how many pulses that will be and enter that number into the pulse number section).  The 10 Hz frequency should produce a constant contraction that is flooding the cytoplasm with calcium for a prolonged period, and in the process activating calcium sensitive kinases.
3. After 20 seconds, decrease the frequency back to 1 Hz and measure the size of the twitches to see if post-tetanic potentiation occurred. 
4. Record the average of three twitches before and after the activation period in Table 5. 
5.  If your twitch tension went down, you could vary the frequency you stimulate the muscle during the activation period, the duration of the activation period, or you can vary both to get an increase in twitch tension after the high frequency stimulation.

<figure>
  <img src= 'img/fig8.png' width='60%'></img>
  <figcaption>**Fig. 8**</figcaption>
</figure>

---

---
Notebook developed by: Alex Nakagawa

Data Science Modules: http://data.berkeley.edu/education/modules
